<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/master/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import requests
from bs4 import BeautifulSoup
import re
baseURL = 'https://www.soccerbase.com'

In [82]:
request = requests.get("https://www.soccerbase.com/players/home.sd")
soup = BeautifulSoup(request.content, "lxml")
testing = soup.find('div', id = 'players-menu')#attrs = {'class': 'soccerColumn soccerColumnLast'})
for div in testing.find_all('div', class_ = 'soccerColumn'):
  if (re.search('Competition', div.text) != None):
    comp = div
threads = []
for num, competition in enumerate(comp.find_all('a'), 1):
  t = threading.Thread()
  print(num, competition['href'])
#for li in testing.find_all('ol'):
#  print(li)

<div class="soccerColumn"><div class="headlineBlock"><h2>Competition</h2></div><ol class="infoList infoListToggle"><li><a href="/tournaments/tournament.sd?comp_id=1">Premier League</a></li><li class="alt"><a href="/tournaments/tournament.sd?comp_id=2">Championship</a></li><li><a href="/tournaments/tournament.sd?comp_id=3">League One</a></li><li class="alt"><a href="/tournaments/tournament.sd?comp_id=4">League Two</a></li></ol><ol class="infoList infoListToggle hide"><li><a href="/tournaments/tournament.sd?comp_id=12">Scottish Premier</a></li><li class="alt"><a href="/tournaments/tournament.sd?comp_id=13">Scottish Championship</a></li><li><a href="/tournaments/tournament.sd?comp_id=14">Scottish League One</a></li><li class="alt"><a href="/tournaments/tournament.sd?comp_id=15">Scottish League Two</a></li></ol><ol class="infoList infoListToggle hide"><li><a href="/tournaments/tournament.sd?comp_id=66">Champions League</a></li><li class="alt"><a href="/tournaments/tournament.sd?comp_id=64"

In [46]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
  return timeIt

In [37]:
#From player page for a team
def getTeamData(team, url, q):
  names = [team]
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
  for elem in test.find_all('b'):
    name = elem.find('a')
    names.append(name.text)
  #return names
  q.put(names)

In [83]:
#From competition page of teams
import threading
import queue
@timer
def readCompetitionPage(compURL):
  request = requests.get(compURL)
  soup = BeautifulSoup(request.content, "lxml")
  teamNames = soup.find('table', class_ = 'table')
  teams = {}
  threads = []
  q = queue.Queue()

  for num, url in enumerate(teamNames.find_all('a'), 1):
    #print(num, url['href'])
    t = threading.Thread(target = getTeamData, args = (url.text, baseURL + url['href'], q))
    threads.append(t)
    t.start()
    #print(url.text)
    #getTeamData(url.text, baseURL+url['href'], q)
    #print(getTeamData(baseURL + url['href']))
  #print(num)
  for t in threads:
    t.join()
  while not q.empty():
    teamData = q.get()
    teams[teamData[0]] = teamData[1:]

  print(teams)
  '''
  for team, players in teams.items():
    print("TEAM:", team, end = '\nPLAYERS: ')
    print(*players, sep=', ')
    print()
'''

In [85]:
readCompetitionPage('https://www.soccerbase.com/tournaments/tournament.sd?comp_id=20')

{'B Leverkusen': ['Lukas Hradecky', 'Lennart Grill', 'Ramazan Ozcan', 'Niklas Lomb', 'Tomasz Kucz', 'Panagiotis Retsos', 'Jonathan Tah', 'Sven Bender', 'Aleksandar Dragovic', 'Lars Bender', 'Edmond Faycial Tapsoba', 'Tin Jedvaj', 'Wendell', 'Daley Sinkgraven', 'Mitchell Weiser', 'Jan Boller', 'Fabian Ruth', 'Kerem Demirbay', 'Nadiem Amiri', 'Julian Baumgartlinger', 'Charles Aranguiz', 'Exequiel Palacios', 'Florian Wirtz', 'Kai Havertz', 'Adrian Stanilewicz', 'Ayman Azhil', 'Herdi Bukusu', 'Paulinho', 'Leon Bailey', 'Lucas Alario', 'Joel Pohjanpalo', 'Moussa Diaby', 'Kevin Volland', 'Karim Bellarabi', 'Florian Wirtz'], 'Wolfsburg': ['Koen Casteels', 'Pavao Pervan', 'Niklas Klinger', 'Lino Kasten', 'Phillip Menzel', 'Michael Edwards', 'William', 'Jeffrey Bruma', 'Paulo Otavio', 'Jerome Roussillon', 'Kevin Mbabu', 'John Brooks', 'Marcel Tisserand', 'Marin Pongracic', 'Tim Siersleben', 'Ignacio Camacho', 'Renato Steffen', 'Yanus Malli', 'Felix Klaus', 'Admir Mehmedi', 'Luca Horn', 'Josua G